In [ ]:
%load_ext autoreload

# Import useful libs

In [ ]:
import os
import sys
import json
import numpy as np
import time
from PIL import Image, ImageDraw
from pathlib import Path
import torch, torchvision
import numpy as np
import os, json, cv2, random
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.data.datasets import register_coco_instances
from detectron2.engine import DefaultTrainer
from detectron2.evaluation import COCOEvaluator

ROOT_DIR = 'F:/detectron2/'
assert os.path.exists(ROOT_DIR)
print(torch.__version__, torch.cuda.is_available())

# Register datasets in COCO format

register_coco_instances(1, 2, 3, 4)

- First parameter (1): the name of the dataset (e.g., 'train', 'test', 'val')
- Second parameter (2): Metadata, you can set it to '{}'
- Third parameter (3): The path to the coco_instances.json file
- Fourth parameter (4): The path to the images folder

In [ ]:
# register train data
register_coco_instances("train", {}, 
                        "F:/0_tents/train/1x/coco_instances.json", 
                        "F:/0_tents/train/1x/amostras")
# register val data
register_coco_instances("val", {}, 
                        "F:/0_tents/val/1x/coco_instances.json", 
                        "F:/0_tents/val/1x/amostras")
#register test data
register_coco_instances("test", {}, 
                        "F:/0_tents/test/1x/coco_instances.json", 
                        "F:/0_tents/test/1x/amostras")

# Trainer

- We have to use a trainer in Detectron2

- We can add new metrics here

- The COCOEvaluator performs the standard COCO metrics

In [ ]:
class CocoTrainer(DefaultTrainer):

  @classmethod
  def build_evaluator(cls, cfg, dataset_name, output_folder=None):

    if output_folder is None:
        os.makedirs("result/Resnet_101", exist_ok=True)
        output_folder = "result/Resnet_101"

    return COCOEvaluator(dataset_name, cfg, False, output_folder)

# Config

Take a look in the Detectron2 defaults.py file. There are many parameters we can change.

This is just a simple example

In [ ]:
from detectron2.engine import DefaultTrainer

# load cfg
cfg = get_cfg()

# Choose model
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_X_101_32x8d_FPN_3x.yaml"))

cfg.DATASETS.TRAIN = ("train",)
cfg.DATASETS.TEST = ("val",)

cfg.DATALOADER.NUM_WORKERS = 0

# Number of images per batch
cfg.SOLVER.IMS_PER_BATCH = 2

# Learning Rate
cfg.SOLVER.BASE_LR = 0.0001

# Number of epochs
cfg.SOLVER.MAX_ITER = 3000

# Steps per learning rate reduction
#cfg.SOLVER.STEPS = (1000,)

# How much learning rate will reduce
#cfg.SOLVER.GAMMA = 0.1

# Number of proposals
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 320   # Default 512

# Number of classes
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # number of classes (only beach straw umbrellas)

# Load pre trained weights
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_X_101_32x8d_FPN_3x.yaml")

# if you have a trained model, comment the line above and use this
#cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")

# backbone freeze, since the images have different number of channels, i think it is best to train all layers
cfg.MODEL.BACKBONE.FREEZE_AT = 0

# Number of epochs to evaluate model, the frequency of evaluation on the val set
cfg.TEST.EVAL_PERIOD = 500

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
#trainer = DefaultTrainer(cfg) # This is another training option
trainer = CocoTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

# This is an evaluation on the val or test set

In [ ]:
from detectron2.data import DatasetCatalog, MetadataCatalog, build_detection_test_loader
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.evaluation import SemSegEvaluator

cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
predictor = DefaultPredictor(cfg)
evaluator = COCOEvaluator("test", cfg, False, output_dir="./output/")
test_loader = build_detection_test_loader(cfg, "test")
inference_on_dataset(trainer.model, test_loader, evaluator)

# Predicting on new images

In [ ]:
from detectron2.utils.visualizer import ColorMode
import glob
import imageio
import cv2
import matplotlib.pyplot as plt

In [ ]:
predictor = DefaultPredictor(cfg) # this is the predictor

In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.DATASETS.TEST = ("test", )
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.6   # set the testing threshold for this model
predictor = DefaultPredictor(cfg)
test_metadata = MetadataCatalog.get("test")

In [ ]:
img_path = 'F:/0_tents/test/1x/amostras/img_1.tiff' # select path to evaluate image

image = imageio.imread(img_path)
print(image.shape)

outputs = predictor(image)
v = Visualizer(image[:,:,1:4]/5.5, metadata=test_metadata, scale=1) # I sliced the image since it should only show RGB for visualization (you can choose any other 3 channels)
out = v.draw_instance_predictions(outputs['instances'].to('cpu'))

plt.figure(figsize=(8,8))
plt.axis('off')
plt.imshow(out.get_image()[:, :, ::-1])

#plt.savefig('fig_43_original', dpi=400,  bbox_inches='tight', pad_inches=0) # saving the image
plt.show()

In [ ]:
outputs # take a look on how the predictions are made